In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns",100)

#Classification
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix,precision_score,recall_score,f1_score

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.preprocessing import normalize,scale

In [2]:
def classification_(X,y): 
    lgbc=LGBMClassifier()
    xgbc=XGBClassifier()
    g=GaussianNB()
    b=BernoulliNB()
    k=KNeighborsClassifier()
    svc=SVC()
    d=DecisionTreeClassifier()
    log=LogisticRegression()
    gbc=GradientBoostingClassifier()
    mn=MultinomialNB()
    rf=RandomForestClassifier()
    ab=AdaBoostClassifier()
    
    algos=[lgbc, xgbc,g,b,k,svc,d,log,gbc,mn,rf,ab]
    algos_name=['LGBMClassifier','XGBClassifier','Gaussian','Bernoulli','KNeighbors','SVC','DecisionTree','LogisticReg','GradientBoosting','MultinominalNB','RandomForest','AdaBoost']

    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.2,random_state=101)

    accuracy = []
    precision = []
    recall = []
    f1 = []
   
    result=pd.DataFrame(columns=['AccuracyScore','PrecisionScore','RecallScore','f1_Score'],index=algos_name)
    
    for i in algos:
        predict=i.fit(X_train,y_train).predict(X_test)
        accuracy.append(accuracy_score(y_test,predict))
        precision.append(precision_score(y_test,predict, average='weighted'))
        recall.append(recall_score(y_test,predict, average='weighted'))
        f1.append(f1_score(y_test,predict, average='weighted'))
      
    result.AccuracyScore=accuracy
    result.PrecisionScore=precision
    result.RecallScore=recall
    result.f1_Score=f1
    
    return result.sort_values('AccuracyScore',ascending=False)

In [3]:
import zipfile
zipfile.ZipFile('../input/whats-cooking/train.json.zip').extractall()
zipfile.ZipFile('../input/whats-cooking/test.json.zip').extractall()
zipfile.ZipFile('../input/whats-cooking/sample_submission.csv.zip').extractall()

In [4]:
df_train = pd.read_json('./train.json')
train_labels = df_train.cuisine
df_train.drop(columns='cuisine', inplace=True)
df_test = pd.read_json('./test.json')
submission = pd.read_csv('./sample_submission.csv')

In [5]:
df_train['ingredients']=df_train.ingredients.apply(lambda x: ' '.join(x))
df_train.head()

,id,ingredients
0,10259,romaine lettuce black olives grape tomatoes ga...
1,25693,plain flour ground pepper salt tomatoes ground...
2,20130,eggs pepper salt mayonaise cooking oil green c...
3,22213,water vegetable oil wheat salt
4,13162,black pepper shallots cornflour cayenne pepper...


In [6]:
df_test['ingredients'] = df_test.ingredients.apply(lambda x: ' '.join(x))
df_test.head()

,id,ingredients
0,18009,baking powder eggs all-purpose flour raisins m...
1,28583,sugar egg yolks corn starch cream of tartar ba...
2,41580,sausage links fennel bulb fronds olive oil cub...
3,29752,meat cuts file powder smoked sausage okra shri...
4,35687,ground black pepper salt sausage casings leeks...


In [7]:
train_labels.head()

0          greek
1    southern_us
2       filipino
3         indian
4         indian
Name: cuisine, dtype: object

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
labels = le.fit_transform(train_labels)
labels

array([ 6, 16,  4, ...,  8,  3, 13])

In [9]:
df = df_train.append(df_test)
df.head()

,id,ingredients
0,10259,romaine lettuce black olives grape tomatoes ga...
1,25693,plain flour ground pepper salt tomatoes ground...
2,20130,eggs pepper salt mayonaise cooking oil green c...
3,22213,water vegetable oil wheat salt
4,13162,black pepper shallots cornflour cayenne pepper...


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(stop_words='english', ngram_range = (1,2), analyzer="word", min_df=0.001)
vect.fit(df['ingredients'])

df_train_dtm=vect.transform(df_train['ingredients'])
train_vect = pd.DataFrame(df_train_dtm.toarray(), columns=vect.get_feature_names_out(), index=df_train.id)
df_test_dtm=vect.transform(df_test['ingredients'])
test_vect = pd.DataFrame(df_test_dtm.toarray(), columns=vect.get_feature_names_out(), index=df_test.id)

In [11]:
train_vect.head()

,active,active dry,added,adobo,adobo sauce,agave,agave nectar,alfredo,alfredo sauce,allspice,allspice ground,almond,almond extract,almond flour,almonds,amchur,american,american cheese,anaheim,anaheim chile,ancho,ancho chile,ancho powder,anchovies,anchovy,anchovy fillets,anchovy paste,andouille,andouille sausage,angel,angel hair,anise,anise seed,apple,apple cider,apple juice,apples,apricot,arbol,arborio,arborio rice,artichok,artichok heart,artichoke,artichoke hearts,artichokes,arugula,asafetida,asafoetida,asiago,...,wild mushrooms,wine,wine carrots,wine chopped,wine dried,wine extra,wine fresh,wine garlic,wine ground,wine olive,wine onions,wine purpose,wine salt,wine scallions,wine vinegar,wings,wonton,wonton wrappers,worcestershire,worcestershire sauce,wrappers,yams,yeast,yeast purpose,yeast salt,yellow,yellow bell,yellow corn,yellow mustard,yellow onion,yellow peppers,yellow squash,yoghurt,yogurt,yogurt fresh,yogurt ground,yogurt salt,yolks,yolks purpose,yolks salt,yolks vanilla,yukon,yukon gold,zest,zest fresh,ziti,zucchini,zucchini fresh,zucchini garlic,zucchini salt
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10259,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25693,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.141668,0.0,0.207169,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.145539,0.0,0.000000,0.0,0.168304,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22213,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13162,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.136143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Deep Learning

In [12]:
from keras.models import Sequential
from keras.layers import Dense
from keras.regularizers import l1
from tensorflow.keras.layers import Dropout

In [13]:
model = Sequential()
model.add(Dense(400,activation='relu', activity_regularizer=l1(0.01)))
model.add(Dense(200,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(100,activation='relu'))
model.add(Dense(200,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(100,activation='relu'))
model.add(Dense(50,activation='relu'))
model.add(Dense(23, activation='softmax'))

2022-08-22 16:14:18.630970: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-22 16:14:18.635834: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-22 16:14:18.636565: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-22 16:14:18.637755: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [14]:
import tensorflow as tf
import keras
model.compile(optimizer='rmsprop', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [15]:
history = model.fit(df_train_dtm.toarray(), labels, validation_split=0.2,epochs = 10, verbose=1)

2022-08-22 16:14:23.732304: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
995/995 [==============================] - 7s 5ms/step - loss: 1.4195 - accuracy: 0.5867 - val_loss: 1.0901 - val_accuracy: 0.6837
Epoch 2/10
995/995 [==============================] - 4s 4ms/step - loss: 0.9291 - accuracy: 0.7433 - val_loss: 0.9780 - val_accuracy: 0.7349
Epoch 3/10
995/995 [==============================] - 4s 4ms/step - loss: 0.7179 - accuracy: 0.8040 - val_loss: 0.9369 - val_accuracy: 0.7589
Epoch 4/10
995/995 [==============================] - 5s 5ms/step - loss: 0.5586 - accuracy: 0.8510 - val_loss: 0.9463 - val_accuracy: 0.7614
Epoch 5/10
995/995 [==============================] - 4s 4ms/step - loss: 0.4393 - accuracy: 0.8867 - val_loss: 1.0433 - val_accuracy: 0.7589
Epoch 6/10
995/995 [==============================] - 4s 4ms/step - loss: 0.3450 - accuracy: 0.9133 - val_loss: 1.0956 - val_accuracy: 0.7578
Epoch 7/10
995/995 [==============================] - 4s 4ms/step - loss: 0.2885 - accuracy: 0.9301 - val_loss: 1.2456 - val_accuracy: 0.7492
Epoch 

## Submission

In [16]:
predictions=model.predict(df_test_dtm.toarray())

In [17]:
classes = predictions.argmax(axis=-1)

In [18]:
pred = le.inverse_transform(classes)

In [19]:
submission['cuisine']=pred.tolist()
submission

,id,cuisine
0,35203,british
1,17600,southern_us
2,35200,italian
3,17602,cajun_creole
4,17605,italian
...,...,...
9939,46537,french
9940,30854,southern_us
9941,15430,italian
9942,46530,southern_us


In [20]:
submission.to_csv('submission.csv', index=False)